In [ ]:
!pip install pyserini
!pip install faiss-cpu
!git clone https://github.com/castorini/anserini.git --recurse-submodules

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 130.5 MB 28 kB/s 
     |████████████████████████████████| 1.5 MB 32.0 MB/s 
     |████████████████████████████████| 13.4 MB 22.3 MB/s 
     |████████████████████████████████| 5.8 MB 38.5 MB/s 
     |████████████████████████████████| 1.3 MB 46.7 MB/s 
     |████████████████████████████████| 4.5 MB 29.5 MB/s 
     |████████████████████████████████| 2.0 MB 51.0 MB/s 
     |████████████████████████████████| 188 kB 59.7 MB/s 
     |████████████████████████████████| 46 kB 2.0 MB/s 
     |████████████████████████████████| 182 kB 53.3 MB/s 
     |████████████████████████████████| 7.6 MB 36.4 MB/s 
     |████████████████████████████████| 86 kB 2.5 MB/s 
  Attempting uninstall: lightgbm
    Found existing installation: lightgbm 2.2.3
    Uninstalling lightgbm-2.2.3:
      Successfully uninstalled lightgbm-2.2.3
Looking in indexes: https://pypi.org/simple, htt

In [ ]:
!wget https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/historical_releases/cord-19_2020-07-16.tar.gz  -P data/cord19_corpus/

--2022-12-05 17:57:58--  https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/historical_releases/cord-19_2020-07-16.tar.gz
Resolving ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com (ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com)... 52.92.197.90, 52.92.193.218, 52.218.160.41, ...
Connecting to ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com (ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com)|52.92.197.90|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3662861028 (3.4G) [binary/octet-stream]
Saving to: ‘data/cord19_corpus/cord-19_2020-07-16.tar.gz.2’

cord-19_2020-07-16. 100%[===================>]   3.41G  34.2MB/s    in 1m 43s  

2022-12-05 17:59:41 (34.0 MB/s) - ‘data/cord19_corpus/cord-19_2020-07-16.tar.gz.2’ saved [3662861028/3662861028]



In [ ]:
!ls data/cord19_corpus/ 
!tar -xvf "/content/data/cord19_corpus/cord-19_2020-07-16.tar.gz" -C "data/cord19_corpus"

2020-07-16		   cord-19_2020-07-16.tar.gz.1	document_parses
cord-19_2020-07-16.tar.gz  cord-19_2020-07-16.tar.gz.2	new_document_parses
2020-07-16/metadata.csv
2020-07-16/document_parses.tar.gz
2020-07-16/changelog
2020-07-16/cord_19_embeddings.tar.gz


In [ ]:
!tar -xvf "/content/data/cord19_corpus/2020-07-16/document_parses.tar.gz" -C "data/cord19_corpus"

Se han truncado las últimas 5000 líneas del flujo de salida.
document_parses/pdf_json/75cc9a3d0d2fc4baea735c4090944046019c8d39.json
document_parses/pmc_json/PMC2778948.xml.json
document_parses/pdf_json/44ffefb264faf5a03e45887a5694fbd07831f303.json
document_parses/pmc_json/PMC7298881.xml.json
document_parses/pdf_json/b5a7acc938d3d05c5ef67cb6b1dd090805e487e5.json
document_parses/pmc_json/PMC4177705.xml.json
document_parses/pdf_json/f620781b4f505d06e58fce2858339994c2f6b812.json
document_parses/pmc_json/PMC7103946.xml.json
document_parses/pdf_json/95528cdd92fb7dcccd02a8fa1294ca69524e1975.json
document_parses/pmc_json/PMC7286232.xml.json
document_parses/pdf_json/b8a7ae4513bce1715847acb4242d6be9396ab974.json
document_parses/pdf_json/230bc38351c778983c30364b4c5d1e725f00b785.json
document_parses/pmc_json/PMC7262020.xml.json
document_parses/pmc_json/PMC7267231.xml.json
document_parses/pdf_json/7991d32f2ab758697ca6d87a726ee826bc0dd5e4.json
document_parses/pmc_json/PMC7088309.xml.json
document_pa

In [ ]:
import json
import os

json_files = os.listdir("/content/data/cord19_corpus/document_parses/pdf_json")

cleaned_data = {}

for i in range(len(json_files)):

  f = open("data/cord19_corpus/document_parses/pdf_json/{}".format(json_files[i]), 'r')
  data = json.load(f)

  if data['abstract'] == []:
    continue

  else:
    cleaned_data['id'] = data['paper_id']
    cleaned_data['contents'] = data['abstract'][0]['text']

    with open("data/cord19_corpus/new_document_parses/{}".format(json_files[i]), "w") as new_json:
      json.dump(cleaned_data, new_json)

In [ ]:
!python -m pyserini.index.lucene -collection JsonCollection -generator DefaultLuceneDocumentGenerator -threads 9 \
-input /content/data/cord19_corpus/new_document_parses -index indexes/lucene-index-msmarco-passage -storePositions -storeDocvectors -storeRaw

Se han truncado las últimas 5000 líneas del flujo de salida.
2022-12-05 18:18:16,651 DEBUG [pool-2-thread-8] index.IndexCollection$LocalIndexerThread (IndexCollection.java:215) - new_document_parses/db7335913f1efeb4a38635bb601f43d04fefe380.json: 1 docs added.
2022-12-05 18:18:16,655 DEBUG [pool-2-thread-4] index.IndexCollection$LocalIndexerThread (IndexCollection.java:215) - new_document_parses/3986351d05e689e73d12d37da43d42c881af449e.json: 1 docs added.
2022-12-05 18:18:16,656 DEBUG [pool-2-thread-5] index.IndexCollection$LocalIndexerThread (IndexCollection.java:215) - new_document_parses/fbdd0aaa4c2a6aa6abad26df74271fab48f15f33.json: 1 docs added.
2022-12-05 18:18:16,656 DEBUG [pool-2-thread-5] index.IndexCollection$LocalIndexerThread (IndexCollection.java:215) - new_document_parses/0a2b82327ae32d76f34a0b8da1ac96c35c8b9af3.json: 1 docs added.
2022-12-05 18:18:16,651 DEBUG [pool-2-thread-2] index.IndexCollection$LocalIndexerThread (IndexCollection.java:215) - new_document_parses/e7c84

In [ ]:
import xml.etree.ElementTree as ET 

# Pass the path of the xml document 
tree = ET.parse('/content/topics-rnd5.xml')

queries = {}

for i in range(50):

  queries[str(i)] = tree.getroot()[i][0].text

queries

{'0': 'coronavirus origin',
 '1': 'coronavirus response to weather changes',
 '2': 'coronavirus immunity',
 '3': 'how do people die from the coronavirus',
 '4': 'animal models of COVID-19',
 '5': 'coronavirus test rapid testing',
 '6': 'serological tests for coronavirus',
 '7': 'coronavirus under reporting',
 '8': 'coronavirus in Canada',
 '9': 'coronavirus social distancing impact',
 '10': 'coronavirus hospital rationing',
 '11': 'coronavirus quarantine',
 '12': 'how does coronavirus spread',
 '13': 'coronavirus super spreaders',
 '14': 'coronavirus outside body',
 '15': 'how long does coronavirus survive on surfaces',
 '16': 'coronavirus clinical trials',
 '17': 'masks prevent coronavirus',
 '18': 'what alcohol sanitizer kills coronavirus',
 '19': 'coronavirus and ACE inhibitors',
 '20': 'coronavirus mortality',
 '21': 'coronavirus heart impacts',
 '22': 'coronavirus hypertension',
 '23': 'coronavirus diabetes',
 '24': 'coronavirus biomarkers',
 '25': 'coronavirus early symptoms',
 '

In [ ]:
from pyserini.index import IndexReader

index_reader = IndexReader('indexes/lucene-index-msmarco-passage')

In [ ]:
index_reader.stats()

{'total_terms': 6386697,
 'documents': 58164,
 'non_empty_documents': 58159,
 'unique_terms': -1}

In [ ]:
len_C = index_reader.stats()['total_terms']

In [ ]:
def len_doc(d):
    doc_vector = index_reader.get_document_vector(d)

    len_d = 0
    for value in doc_vector.values():
      len_d += value
    
    return len_d

In [ ]:
def coll_freq(t):
  
    cf = index_reader.get_term_counts(t)[1]

    return cf

In [ ]:
def term_freq(t, d):
    doc = index_reader.get_document_vector(d)
    
    if t in doc: tf = doc[t]
    else: tf = 0
    
    return tf

In [ ]:
def prob_t_Md(t, d, lambd):

    p_t_Md = (1-lambd)*(term_freq(t, d) / abs(len_doc(d))) + lambd*(coll_freq(t) / abs(len_C))

    return p_t_Md


In [ ]:
import math

def score_doc(q, d, lambd):

    p_q_Md_total = 0

    for term in q:

      p_q_Md_single = prob_t_Md(term, d, lambd)

      if p_q_Md_single == 0:
        continue

      else:
        p_q_Md_total += math.log(p_q_Md_single) 

    return p_q_Md_total
         

In [ ]:
from pyserini.search.lucene import LuceneSearcher

results = []
searcher = LuceneSearcher("indexes/lucene-index-msmarco-passage")

lambd = 0.1

for qid, q in queries.items():

    hits = searcher.search(q, k=100)
    query = index_reader.analyze(q)

    for hit in hits:
      final_score = score_doc(query, hit.docid, lambd)
      results.append([qid, hit.docid, final_score])   

In [ ]:
check = set()
for res in results:
    if ((res[0], res[1])) in check:
        raise Exception("Error: Duplicate query-doc is found", res[0], res[1])
    check.add((res[0], res[1]))

# write results in a file
output_str = "\n".join([l[0] + "\tQ0\t" + l[1] + "\t0\t" + str(l[2]) + "\tlm_jm" for l in results])
open("\content\lm_jm.run", "w").write(output_str)

371420